<a href="https://colab.research.google.com/github/yukinaga/bert_nlp/blob/main/section_2/03_simple_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# シンプルなBERTの実装
訓練済みのモデルを使用し、文章の一部の予測、及び2つの文章が連続しているかどうかの判定を行います。

## ライブラリのインストール
PyTorch-Transformers、および必要なライブラリのインストールを行います。

In [1]:
!pip install folium==0.2.1
!pip install urllib3==1.25.11
!pip install pytorch-transformers==1.2.0

## 文章の一部の予測
文章における一部の単語をMASKし、それをBERTのモデルを使って予測します。

In [2]:
import torch
from pytorch_transformers import BertForMaskedLM
from pytorch_transformers import BertTokenizer
#今回は設定しないのでconfigは不要

text = "[CLS] I played baseball with my friends at school yesterday [SEP]"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
words = tokenizer.tokenize(text)#textsをwordsに変換
print(words)

['[CLS]', 'i', 'played', 'baseball', 'with', 'my', 'friends', 'at', 'school', 'yesterday', '[SEP]']


文章の一部をMASKします。

In [3]:
msk_idx = 3
words[msk_idx] = "[MASK]"  # msk_idxの単語(baseball)を[MASK]に置き換える
print(words)

['[CLS]', 'i', 'played', '[MASK]', 'with', 'my', 'friends', 'at', 'school', 'yesterday', '[SEP]']


単語を対応するインデックスに変換します。

In [50]:
word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
print(type(word_ids))
word_tensor = torch.tensor([word_ids])  # テンソルに変換
print(word_tensor)

<class 'list'>
tensor([[ 101, 1045, 2209,  103, 2007, 2026, 2814, 2012, 2082, 7483,  102]])


BERTのモデルを使って予測を行います。

In [47]:
msk_model = BertForMaskedLM.from_pretrained("bert-base-uncased")
# msk_model.cuda()  # GPU対応
msk_model.eval()#評価モード

# x = word_tensor.cuda()  # GPU対応
x = word_tensor  # xとしてid化したwordsを与える。GPUなし
y = msk_model(x)  # 順伝搬を1回行って予測
print('y:',y)
result = y[0]#tensorの要素はバッチ毎なので、0バッチ目をスライス
print('y[0]:',result)
print('y[0].size():',result.size())  # 結果の形状:1,11,30522 1:バッチサイズ 11:学習した文章のサイズ 30522:全トークン数

_, max_ids = torch.topk(result[0][msk_idx], k=5)  # 最も大きい5つの値と、その大きい値のインデックス
print('_:',_)
print('max_ids:',max_ids)
result_words = tokenizer.convert_ids_to_tokens(max_ids.tolist())  # インデックスを単語に変換
print(result_words)

y: (tensor([[[ -6.6873,  -6.6405,  -6.6409,  ...,  -6.0201,  -5.8183,  -3.9777],
         [ -9.5150,  -9.3415,  -9.3818,  ...,  -8.4236,  -8.4428,  -5.3152],
         [-10.0567, -10.1768, -10.2753,  ...,  -8.5044,  -8.6216,  -5.3011],
         ...,
         [-13.6662, -14.2769, -13.8572,  ..., -12.8681, -11.8016, -11.4663],
         [ -9.2015,  -8.9383,  -9.3056,  ...,  -7.7869,  -9.2608,  -3.0500],
         [-13.1242, -12.9604, -12.7900,  ...,  -9.9769, -10.1773, -10.8939]]],
       grad_fn=<AddBackward0>),)
y[0]: tensor([[[ -6.6873,  -6.6405,  -6.6409,  ...,  -6.0201,  -5.8183,  -3.9777],
         [ -9.5150,  -9.3415,  -9.3818,  ...,  -8.4236,  -8.4428,  -5.3152],
         [-10.0567, -10.1768, -10.2753,  ...,  -8.5044,  -8.6216,  -5.3011],
         ...,
         [-13.6662, -14.2769, -13.8572,  ..., -12.8681, -11.8016, -11.4663],
         [ -9.2015,  -8.9383,  -9.3056,  ...,  -7.7869,  -9.2608,  -3.0500],
         [-13.1242, -12.9604, -12.7900,  ...,  -9.9769, -10.1773, -10.8939]]],
 

In [48]:
print('result[0][3]:',result[0][3])  # 単語をインデックスに変換
print('result[0][3].argmax():',result[0][3].argmax())
print('idからwordに変換',tokenizer.convert_ids_to_tokens(result[0][3].argmax().item()))

result[0][3]: tensor([-5.3318, -5.5466, -5.5536,  ..., -5.1736, -5.8427, -4.8701],
       grad_fn=<SelectBackward>)
result[0][3].argmax(): tensor(3455)
idからwordに変換 basketball


## 文章が連続しているかどうかの判定
BERTのモデルを使って、2つの文章が連続しているかどうかの判定を行います。  
以下の関数`show_continuity`では、2つの文章の連続性を判定し、表示します。

In [52]:
from pytorch_transformers import BertForNextSentencePrediction

def show_continuity(text, seg_ids):
    words = tokenizer.tokenize(text)#単語分割
    word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
    word_tensor = torch.tensor([word_ids])  # テンソルに変換

    seg_tensor = torch.tensor([seg_ids])

    nsp_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
    # nsp_model.cuda()  # GPU対応
    nsp_model.eval()#評価モード

    # x = word_tensor.cuda()  # GPU対応
    x = word_tensor  # GPUなし
    # s = seg_tensor.cuda()  # GPU対応
    s = seg_tensor  # GPUなし

    y = nsp_model(x, s)  # 予測
    result = torch.softmax(y[0], dim=1)#resultをsoftmaxに入れ、確率にする
    print(result)
    print(str(result[0][0].item()*100) + "%の確率で連続しています。")

`show_continuity`関数に、自然につながる2つの文章を与えます。

In [56]:
text = "[CLS] What is baseball ? [SEP] It is a game of hitting the ball with the bat [SEP]"
print(tokenizer.tokenize(text))
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1]  # 0:前の文章([CLS]から[SEP]まで)の単語、1:後の文章の単語
print(seg_ids)
show_continuity(text, seg_ids)

['[CLS]', 'what', 'is', 'baseball', '?', '[SEP]', 'it', 'is', 'a', 'game', 'of', 'hitting', 'the', 'ball', 'with', 'the', 'bat', '[SEP]']
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tensor([[1.0000e+00, 4.5869e-06]], grad_fn=<SoftmaxBackward>)
99.9995470046997%の確率で連続しています。


`show_continuity`関数に、自然につながらない2つの文章を与えます。

In [57]:
text = "[CLS] What is baseball ? [SEP] This food is made with flour and milk [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)

tensor([[9.5296e-06, 9.9999e-01]], grad_fn=<SoftmaxBackward>)
0.0009529647286399268%の確率で連続しています。


In [1]:
dir(str())

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'removeprefix',
 'removesuffix',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
